In [2]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hlh
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
import_lustre_dir="file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/hail-20211105-1046-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [6]:
path=import_lustre_dir

objects = hl.utils.hadoop_ls(path)
vcf_header=f"{lustre_dir}/vcf_header.txt"
#print(objects)
print("Reading vcf filenames")
    #vcfs = [vcf["path"] for vcf in objects if (vcf["path"].startswith(f"{s3location_input}chr"+chromosome+"_") and vcf["path"].endswith(".vcf.gz"))]
vcfs = [vcf["path"] for vcf in objects if (vcf["path"].startswith(f"{path}/ukbb") and vcf["path"].endswith(".vcf.gz"))]

print(vcfs)

mt = hl.import_vcf(vcfs, array_elements_required=False, 
                       force_bgz=True, header_file= vcf_header)
print("Imported vcf files")
   

mt.write(f"{lustre_dir}/ukbb_hits.mt")

Reading vcf filenames
['file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_757.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_939.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_704.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_782.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_192.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_894.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_190.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_354.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_866.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_961.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_936.vcf.gz', 'file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge

2021-11-05 10:50:06 Hail: WARN: Loading user-provided header file. The sample IDs, INFO fields, and FORMAT fields were not checked for agreement with input data.


Imported vcf files


2021-11-05 10:50:15 Hail: INFO: Coerced sorted dataset
2021-11-05 10:50:15 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-11-05 10:50:24 Hail: INFO: wrote matrix table with 47 rows and 200643 columns in 20 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_hits.mt
    Total size: 31.98 MiB
    * Rows/entries: 30.85 MiB
    * Columns: 1.13 MiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (383.71 KiB)
    * Largest partition:  18 rows (12.85 MiB)


In [7]:
path=import_lustre_dir

objects = hl.utils.hadoop_ls(path)
vcf_header=f"{lustre_dir}/ibd_header.txt"
#print(objects)
print("Reading vcf filenames")
    #vcfs = [vcf["path"] for vcf in objects if (vcf["path"].startswith(f"{s3location_input}chr"+chromosome+"_") and vcf["path"].endswith(".vcf.gz"))]
vcfs = [vcf["path"] for vcf in objects if (vcf["path"].startswith(f"{path}/IBD") and vcf["path"].endswith(".vcf.gz"))]

print(vcfs)

mt = hl.import_vcf(vcfs, array_elements_required=False, 
                       force_bgz=True, header_file= vcf_header)
print("Imported vcf files")
   

mt.write(f"{lustre_dir}/ibd_hits.mt")

Reading vcf filenames
['file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/IBD_hits.vcf.gz']
Imported vcf files


2021-11-05 10:56:21 Hail: INFO: Coerced sorted dataset
2021-11-05 10:56:24 Hail: INFO: wrote matrix table with 46 rows and 21465 columns in 1 partition to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ibd_hits.mt
    Total size: 3.58 MiB
    * Rows/entries: 3.50 MiB
    * Columns: 84.29 KiB
    * Globals: 11.00 B
    * Smallest partition: 46 rows (3.50 MiB)
    * Largest partition:  46 rows (3.50 MiB)


In [8]:
mt1=hl.read_matrix_table(f"{lustre_dir}/ukbb_hits.mt")
mt2=hl.read_matrix_table(f"{lustre_dir}/ibd_hits.mt")

In [10]:
mt=mt1.union_cols(mt2)

In [14]:
mt1.count()

(47, 200643)

In [15]:
mt2.count()

(46, 21465)

In [13]:
mt.count()

(34, 222108)

In [16]:
mt.write(f"{lustre_dir}/intersection_hits.mt")

2021-11-05 10:59:17 Hail: INFO: wrote matrix table with 34 rows and 222108 columns in 20 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/intersection_hits.mt
    Total size: 26.09 MiB
    * Rows/entries: 24.88 MiB
    * Columns: 1.21 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  14 rows (10.94 MiB)


In [18]:
mt.summarize()

2021-11-05 11:00:33 Hail: INFO: Coerced sorted dataset


Non-missing,222108 (100.00%)
Missing,0
Min Size,7
Max Size,15
Mean Size,7.77
Sample Values,"['-000001', '-000002', '-000003', '-000004', '-000005']"


Non-missing,34 (100.00%)
Missing,0
Contig Counts,"{'chr11': 2, 'chr5': 2, 'chr22': 1, 'chr19': 2, 'chr1': 2, 'chr12': 2, 'chr2': 3, 'chr17': 1, 'chr4': 2, 'chr6': 1, 'chr9': 2, 'chr16': 14}"
Non-missing,34 (100.00%)
Missing,0
Min Size,2
Max Size,3
Mean Size,2.09
Non-missing,71 (100.00%)
Missing,0
Min Size,1


Non-missing,7550325 (99.98%)
Missing,1347 (0.02%)
Homozygous Reference,7100267
Heterozygous,434592
Homozygous Variant,15466
Ploidy,{2: 7550325}
Phased,{False: 7550325}
Non-missing,7551672 (100.00%)
Missing,0
Min Size,1
Max Size,1


In [19]:
mt.s.show()

""
s
str
"""5246185"""
"""4271618"""
"""2516301"""
"""4016248"""
"""4794472"""
"""3377156"""
"""4816637"""
"""1082673"""


In [23]:
path=import_lustre_dir

ibd_vcf=f"{import_lustre_dir}/ibd_akt.vcf.gz"
ukbb_vcf=f"{import_lustre_dir}/ukb_akt.sorted.vcf.gz"


mt1 = hl.import_vcf(ibd_vcf, reference_genome='GRCh38', force_bgz=True, array_elements_required=False)
mt2 = hl.import_vcf(ukbb_vcf, reference_genome='GRCh38', force_bgz=True, array_elements_required=False)


print("Imported vcf files")
   

mt1.write(f"{lustre_dir}/ibd_akt.mt", overwrite=True)
mt2.write(f"{lustre_dir}/ukb_akt.mt", overwrite=True)

Imported vcf files


2021-11-05 15:17:31 Hail: INFO: Coerced sorted dataset
2021-11-05 15:18:28 Hail: INFO: wrote matrix table with 30788 rows and 21465 columns in 21 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ibd_akt.mt
    Total size: 3.19 GiB
    * Rows/entries: 3.19 GiB
    * Columns: 84.29 KiB
    * Globals: 11.00 B
    * Smallest partition: 1452 rows (155.45 MiB)
    * Largest partition:  1475 rows (156.11 MiB)
2021-11-05 15:18:37 Hail: INFO: Coerced sorted dataset
2021-11-05 15:20:19 Hail: INFO: wrote matrix table with 28769 rows and 200643 columns in 183 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukb_akt.mt
    Total size: 26.98 GiB
    * Rows/entries: 26.98 GiB
    * Columns: 1.13 MiB
    * Globals: 11.00 B
    * Smallest partition: 151 rows (149.38 MiB)
    * Largest partition:  165 rows (153.40 MiB)


In [24]:
mt1=hl.read_matrix_table(f"{lustre_dir}/ukbb_hits.mt")
mt2=hl.read_matrix_table(f"{lustre_dir}/ibd_hits.mt")
mt1_akt=hl.read_matrix_table(f"{lustre_dir}/ukb_akt.mt")
mt2_akt=hl.read_matrix_table(f"{lustre_dir}/ibd_akt.mt")

In [25]:
mt_ukb=mt1.union_rows(mt1_akt)
mt_ibd=mt2.union_rows(mt2_akt)

In [30]:
mt_ukb.count()

(28816, 200643)

In [31]:
mt_ibd.count()

(30834, 21465)

In [26]:
mt=mt_ukb.union_cols(mt_ibd)

In [27]:
mt.write(f"{lustre_dir}/ukb_ibd_akt_hits_intersection.mt", overwrite=True)

2021-11-05 15:26:14 Hail: INFO: wrote matrix table with 23865 rows and 222108 columns in 223 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukb_ibd_akt_hits_intersection.mt
    Total size: 24.70 GiB
    * Rows/entries: 24.70 GiB
    * Columns: 1.21 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  147 rows (150.60 MiB)


In [28]:
mt=hl.read_matrix_table(f"{lustre_dir}/ukb_ibd_akt_hits_intersection.mt")

In [29]:
mt.count()

(23865, 222108)